# Profiling a dataset with pandas

In [1]:
import re
import pandas as pd
import math
import datetime
from collections import Counter

data = {
    'A': ['abc123', 'def/456', 'ghi-789', 'jkl101', 'AAA', '111', None, 'vwx145', 'yz156', 'lmn167'],
    'B': [1, 1, 3, 4, 5, 5, 7, 8, 9, 9],
    'C': ['foo', 1, 'foo', 'bar', 15, 'bar', 'foo', "2023/01/10", 'foo', 'bar']
}
df = pd.DataFrame(data)

## Get the data pattern

In [2]:
REGEX_CARS = r'[a-zA-Z]'
REGEX_NUMS = r'\d'
REGEX_NOISE = r'[µ&#@^~]'
REGEX_SPECCARS = r'[<>µ+=*&\'"#{}()|/\\@][]^~]'
REGEX_PONCT = r'[,.;:?!]'

def getStringPattern(string):
    if (string == 'nan' or string == None):
        return "NULL"
    mystr = re.sub(REGEX_CARS, 'C', string)
    mystr = re.sub(REGEX_NUMS, 'N', mystr)
    mystr = re.sub(REGEX_NOISE, '?', mystr)
    return mystr

In [3]:
df2 = pd.DataFrame()
df2['profile'] = df['A'].apply(getStringPattern)
df2

,profile
0,CCCNNN
1,CCC/NNN
2,CCC-NNN
3,CCCNNN
4,CCC
5,NNN
6,NULL
7,CCCNNN
8,CCNNN
9,CCCNNN


Putting the result in a JSON format ...

In [4]:
df2['profile'].value_counts().to_dict()

{'CCCNNN': 4,
 'CCC/NNN': 1,
 'CCC-NNN': 1,
 'CCC': 1,
 'NNN': 1,
 'NULL': 1,
 'CCNNN': 1}

## Discovering the real type

In [5]:
def getType(value):
	"""Returns the data type on the value in input
	Args:
		value (object): data
	Returns:
		str: type namen can be [ null, number, date, string, unknown]
	"""
	if value is None:
		return "null"
	if isinstance(value, (int, float)):
		if math.isnan(value):
			return "null"
		else:
			return "number"
	elif isinstance(value, str):
		return "string"
	elif isinstance(value, datetime.datetime):
		return "date"
	else:
		return "unknown"

In [6]:
df2 = pd.DataFrame()
df2['types'] = df['C'].apply(getType)
df2

,types
0,string
1,number
2,string
3,string
4,number
5,string
6,string
7,string
8,string
9,string


The problem is if a date is stored in a string format, cannot detect it !

In [7]:
isinstance("2023/01/10", datetime.datetime)

False

We can use the dateutil package to check that ...

In [8]:
from dateutil.parser import parse

def is_date(string, fuzzy=False):
    try:
        parse(string, fuzzy=fuzzy)
        return True
    except ValueError:
        return False

In [9]:
is_date("2023/01/10")

True

So the function should now become:

In [10]:
def getType(value):
	"""Returns the data type on the value in input
	Args:
		value (object): data
	Returns:
		str: type namen can be [ null, number, date, string, unknown]
	"""
	if value is None:
		return "null"
	if isinstance(value, (int, float)):
		if math.isnan(value):
			return "null"
		else:
			return "number"
	elif isinstance(value, str):
		try:
			parse(value, fuzzy=False)
			return "date"
		except ValueError:
			return "string"
	elif isinstance(value, datetime.datetime):
		return "date"
	else:
		return "unknown"

In [11]:
df2 = pd.DataFrame()
df2['types'] = df['C'].apply(getType)
df2

,types
0,string
1,number
2,string
3,string
4,number
5,string
6,string
7,date
8,string
9,string


Get stats from that result

In [12]:
df2.value_counts() / df2.shape[0]  * 100

types 
string    70.0
number    20.0
date      10.0
dtype: float64

## Global profile function

In [13]:
def profile(df, maxvaluecounts=10) -> dict:
	"""Build a JSON which contains some basic profiling informations
	Args:
		maxvaluecounts (int, optional): Limits the number of value_counts() return. Defaults to 10.
	Returns:
		json: data profile in a JSON format
	"""
	profile = {}
	# Get stats per columns/fields
	profileColumns = []
	for col in df.columns:
		profileCol = {}
		counts = df[col].value_counts()
		profileCol['name'] = col
		profileCol['type'] = str(df[col].dtypes)
		profileCol['inferred'] = str(df[col].infer_objects().dtypes)
		profileCol['distinct'] = int(len(counts))
		profileCol['nan'] = int(df[col].isna().sum())
		profileCol['null'] = int(df[col].isnull().sum())
		profileCol['stats'] = df[col].describe().to_dict()
		profileCol['top values'] = dict(Counter(counts.to_dict()).most_common(maxvaluecounts))
		# Get pattern for that column
		dfProfPattern = pd.DataFrame()
		dfProfPattern['profile'] = df[col].apply(lambda x:getStringPattern(str(x)))
		profileCol['pattern'] = dict(Counter(dfProfPattern['profile'].value_counts().to_dict()).most_common(maxvaluecounts))
		# get types for that columns
		dfProfType = pd.DataFrame()
		dfProfType['types'] = df[col].apply(lambda x:getType(x))
		profileCol['types'] = dict(Counter(dfProfType['types'].value_counts().to_dict()).most_common(maxvaluecounts))
		profileColumns.append(profileCol)
	profile["rows count"] = df.shape[0]
	profile["columns count"] = len(df.columns)
	profile["columns names"] = [ name for name in df.columns ]
	profile["columns"] = profileColumns
	return profile

In [14]:
df.head(10)

,A,B,C
0,abc123,1,foo
1,def/456,1,1
2,ghi-789,3,foo
3,jkl101,4,bar
4,AAA,5,15
5,111,5,bar
6,None,7,foo
7,vwx145,8,2023/01/10
8,yz156,9,foo
9,lmn167,9,bar


In [15]:
profile(df)

{'rows count': 10,
 'columns count': 3,
 'columns names': ['A', 'B', 'C'],
 'columns': [{'name': 'A',
   'type': 'object',
   'inferred': 'object',
   'distinct': 9,
   'nan': 1,
   'null': 1,
   'stats': {'count': 9, 'unique': 9, 'top': 'abc123', 'freq': 1},
   'top values': {'abc123': 1,
    'def/456': 1,
    'ghi-789': 1,
    'jkl101': 1,
    'AAA': 1,
    '111': 1,
    'vwx145': 1,
    'yz156': 1,
    'lmn167': 1},
   'pattern': {'CCCNNN': 4,
    'CCC/NNN': 1,
    'CCC-NNN': 1,
    'CCC': 1,
    'NNN': 1,
    'CCCC': 1,
    'CCNNN': 1},
   'types': {'string': 8, 'date': 1, 'null': 1}},
  {'name': 'B',
   'type': 'int64',
   'inferred': 'int64',
   'distinct': 7,
   'nan': 0,
   'null': 0,
   'stats': {'count': 10.0,
    'mean': 5.2,
    'std': 3.011090610836324,
    'min': 1.0,
    '25%': 3.25,
    '50%': 5.0,
    '75%': 7.75,
    'max': 9.0},
   'top values': {1: 2, 5: 2, 9: 2, 3: 1, 4: 1, 7: 1, 8: 1},
   'pattern': {'N': 10},
   'types': {'number': 10}},
  {'name': 'C',
   'type'